##1. 라이브러리 및 패키지 불러오기

In [ ]:
import numpy as np # 행렬 연산 / 데이터 핸들링
import pandas as pd # 데이터 분석
import matplotlib.pyplot as plt # 그래프 시각화
import seaborn as sns # 그래프 시각화

In [ ]:
from sklearn.preprocessing import LabelEncoder # 라벨 인코더
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score, roc_curve, precision_recall_curve, precision_score, recall_score, auc
from sklearn.metrics import confusion_matrix, plot_confusion_matrix
from sklearn.metrics import classification_report, accuracy_score, f1_score
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler

In [ ]:
respec = pd.read_csv('/content/drive/MyDrive/2022_Bigcontest/Data/respec.csv')
respec.head()

,Unnamed: 0,application_id,user_id,gender,insert_time,credit_score,yearly_income,income_type,employment_type,houseown_type,desired_amount,purpose,personal_rehabilitation_yn,personal_rehabilitation_complete_yn,existing_loan_cnt,existing_loan_amt,reage,career
0,1,954900,553686,1.0,2022-06-07 14:29:03,870.0,30000000.0,1,정규직,기타가족소유,30000000.0,SWITCHLOAN,0.0,1.0,1.0,27000000.0,50.0,16.0
1,2,137274,59516,1.0,2022-06-07 21:40:22,710.0,30000000.0,2,기타,기타가족소유,10000000.0,LIVING,1.0,1.0,5.0,15000000.0,20.0,2.0
2,3,1570936,167320,1.0,2022-06-07 09:40:27,820.0,62000000.0,3,정규직,자가,2000000.0,LIVING,1.0,1.0,7.0,344000000.0,30.0,6.0
3,4,967833,33400,1.0,2022-06-07 08:55:07,630.0,36000000.0,3,정규직,기타가족소유,5000000.0,LIVING,0.0,0.0,1.0,16000000.0,20.0,2.0
4,6,1614045,860410,1.0,2022-06-07 11:29:05,590.0,45000000.0,4,기타,자가,15000000.0,SWITCHLOAN,1.0,0.0,1.0,20000000.0,40.0,6.0


In [ ]:
respec = respec.drop(['Unnamed: 0'], axis = 1)

# 2. 전처리

In [ ]:
respec_final = respec

In [ ]:
respec = respec.drop(['application_id', 'user_id', 'insert_time'], axis = 1)

In [ ]:
respec_train = respec[respec['credit_score'].notnull()]

In [ ]:
respec_test = respec[respec['credit_score'].isnull()]

In [ ]:
# 인코딩
respec_train = pd.get_dummies(respec_train)
respec_test= pd.get_dummies(respec_test)

In [ ]:
train_columns = respec_train.columns
test_columns = respec_test.columns

In [ ]:
respec_train = respec_train.values.astype(float)
respec_test = respec_test.values.astype(float)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
min_max_scaler = MinMaxScaler()

In [ ]:
respec_train = min_max_scaler.fit_transform(respec_train)
respec_test = min_max_scaler.fit_transform(respec_test)

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_data.py:461: RuntimeWarning: All-NaN slice encountered
  data_min = np.nanmin(X, axis=0)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_data.py:462: RuntimeWarning: All-NaN slice encountered
  data_max = np.nanmax(X, axis=0)


In [ ]:
respec_train = pd.DataFrame(respec_train, columns = train_columns)
respec_test = pd.DataFrame(respec_test, columns = test_columns)

In [ ]:
data = respec_train.drop('credit_score',axis=1)
target = respec_train['credit_score']

In [ ]:
standardScaler = StandardScaler()
print(standardScaler.fit(data))
data_scaled = standardScaler.transform(data)

StandardScaler()


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(data_scaled, target, test_size=0.3, random_state=0)

## 2) 적용

In [ ]:
test_data = respec_test.drop('credit_score',axis=1)
target = respec_test['credit_score']

In [ ]:
standardScaler = StandardScaler()
print(standardScaler.fit(test_data))
data_scaled = standardScaler.transform(test_data)

StandardScaler()


# 3. XGBoost

##1) 파라미터 X

In [ ]:
import xgboost

xgb_reg = xgboost.XGBRegressor()
xgb_reg.fit(X_train, y_train)
y_pred = xgb_reg.predict(X_val)

[12:05:41] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [ ]:
xgb_reg.fit(X_train,y_train,eval_set = [(X_val, y_val)], early_stopping_rounds = 2)
y_pred = xgb_reg.predict(X_val)

[12:06:38] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation_0-rmse:0.181619
Will train until validation_0-rmse hasn't improved in 2 rounds.
[1]	validation_0-rmse:0.169207
[2]	validation_0-rmse:0.158429
[3]	validation_0-rmse:0.149085
[4]	validation_0-rmse:0.141046
[5]	validation_0-rmse:0.134131
[6]	validation_0-rmse:0.128209
[7]	validation_0-rmse:0.123204
[8]	validation_0-rmse:0.118948
[9]	validation_0-rmse:0.115283
[10]	validation_0-rmse:0.11223
[11]	validation_0-rmse:0.109656
[12]	validation_0-rmse:0.107502
[13]	validation_0-rmse:0.10565
[14]	validation_0-rmse:0.104151
[15]	validation_0-rmse:0.102721
[16]	validation_0-rmse:0.101509
[17]	validation_0-rmse:0.100554
[18]	validation_0-rmse:0.099667
[19]	validation_0-rmse:0.098946
[20]	validation_0-rmse:0.098305
[21]	validation_0-rmse:0.09773
[22]	validation_0-rmse:0.09732
[23]	validation_0-rmse:0.096926
[24]	validation_0-rmse:0.096565
[25]	validation_0-rmse:

In [ ]:
mse = mean_squared_error(y_val, y_pred)
rmse = np.sqrt(mse)

print('MSE : {0:.3f}, RMSE : {1:.3F}'.format(mse , rmse))
print('Variance score : {0:.3f}'.format(r2_score(y_val,y_pred)))

MSE : 0.009, RMSE : 0.092
Variance score : 0.421


##2) 파라미터 ver.1


In [ ]:
import xgboost

model = xgboost.XGBRegressor(n_estimators=500, learning_rate=0.3, max_depth=10, random_state = 2022)
model.fit(X_train, y_train)
y_pred = model.predict(X_val) 

In [ ]:
mse = mean_squared_error(y_val, y_pred)
rmse = np.sqrt(mse)

print('MSE : {0:.3f}, RMSE : {1:.3F}'.format(mse , rmse))
print('Variance score : {0:.3f}'.format(r2_score(y_val,y_pred)))

MSE : 0.006, RMSE : 0.077
Variance score : 0.599


In [ ]:
import xgboost

model = xgboost.XGBRegressor(n_estimators=500, learning_rate=0.3, max_depth=10, random_state = 2022)
model.fit(X_train, y_train)
y_pred = model.predict(X_val) 

[01:05:00] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [ ]:
mse = mean_squared_error(y_val, y_pred)
rmse = np.sqrt(mse)

print('MSE : {0:.3f}, RMSE : {1:.3F}'.format(mse , rmse))
print('Variance score : {0:.3f}'.format(r2_score(y_val,y_pred)))

MSE : 0.006, RMSE : 0.077
Variance score : 0.599


In [ ]:
import xgboost

model = xgboost.XGBRegressor(n_estimators=700, learning_rate=0.3, max_depth=10, random_state = 2022)
model.fit(X_train, y_train)
y_pred = model.predict(X_val) 

[01:20:36] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [ ]:
mse = mean_squared_error(y_val, y_pred)
rmse = np.sqrt(mse)

print('MSE : {0:.3f}, RMSE : {1:.3F}'.format(mse , rmse))
print('Variance score : {0:.3f}'.format(r2_score(y_val,y_pred)))

MSE : 0.006, RMSE : 0.076
Variance score : 0.611


In [ ]:
import xgboost

model = xgboost.XGBRegressor(n_estimators=1000, learning_rate=0.3, max_depth=10, random_state = 2022)
model.fit(X_train, y_train)
y_pred = model.predict(X_val)

[02:04:14] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [ ]:
mse = mean_squared_error(y_val, y_pred)
rmse = np.sqrt(mse)

print('MSE : {0:.3f}, RMSE : {1:.3F}'.format(mse , rmse))
print('Variance score : {0:.3f}'.format(r2_score(y_val,y_pred)))

MSE : 0.006, RMSE : 0.075
Variance score : 0.621


In [ ]:
import xgboost

model = xgboost.XGBRegressor(n_estimators=1000, learning_rate=0.1, max_depth=10, random_state = 2022)
model.fit(X_train, y_train)
y_pred = model.predict(X_val)

[03:09:26] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [ ]:
mse = mean_squared_error(y_val, y_pred)
rmse = np.sqrt(mse)

print('MSE : {0:.3f}, RMSE : {1:.3F}'.format(mse , rmse))
print('Variance score : {0:.3f}'.format(r2_score(y_val,y_pred)))

MSE : 0.006, RMSE : 0.078
Variance score : 0.588


In [ ]:
import xgboost

model = xgboost.XGBRegressor(n_estimators=1000, learning_rate=0.3, max_depth=12, random_state = 2022)
model.fit(X_train, y_train)
y_pred = model.predict(X_val)

[04:09:31] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [ ]:
mse = mean_squared_error(y_val, y_pred)
rmse = np.sqrt(mse)

print('MSE : {0:.3f}, RMSE : {1:.3F}'.format(mse , rmse))
print('Variance score : {0:.3f}'.format(r2_score(y_val,y_pred)))

MSE : 0.005, RMSE : 0.074
Variance score : 0.633


##3) 파라미터 튜닝

In [ ]:
xgb_model = xgboost.XGBRegressor()

# 후보 파라미터 선정
params = {'max_depth':[12, 14], 'min_child_weight':[1,3], 'colsample_bytree':[0.5,0.75], 'learning_rate' : [0.3, 0.4]}
#{'colsample_bytree': 0.75, 'learning_rate': 0.3, 'max_depth': 14, 'min_child_weight': 1}

# gridsearchcv 객체 정보 입력(어떤 모델, 파라미터 후보, 교차검증 몇 번)
gridcv = GridSearchCV(xgb_model, param_grid=params, cv=3)

# 파라미터 튜닝 시작
gridcv.fit(X_train, y_train, early_stopping_rounds=30, eval_metric='rmse', eval_set=[(X_val, y_val)])

#튜닝된 파라미터 출력
print(gridcv.best_params_)

[04:52:26] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation_0-rmse:0.152552
Will train until validation_0-rmse hasn't improved in 30 rounds.
[1]	validation_0-rmse:0.128579
[2]	validation_0-rmse:0.111473
[3]	validation_0-rmse:0.102157
[4]	validation_0-rmse:0.097064
[5]	validation_0-rmse:0.09415
[6]	validation_0-rmse:0.09189
[7]	validation_0-rmse:0.090603
[8]	validation_0-rmse:0.089558
[9]	validation_0-rmse:0.088809
[10]	validation_0-rmse:0.088229
[11]	validation_0-rmse:0.087847
[12]	validation_0-rmse:0.087726
[13]	validation_0-rmse:0.087667
[14]	validation_0-rmse:0.08756
[15]	validation_0-rmse:0.087237
[16]	validation_0-rmse:0.087118
[17]	validation_0-rmse:0.086941
[18]	validation_0-rmse:0.086832
[19]	validation_0-rmse:0.086613
[20]	validation_0-rmse:0.086492
[21]	validation_0-rmse:0.086287
[22]	validation_0-rmse:0.086235
[23]	validation_0-rmse:0.086229
[24]	validation_0-rmse:0.086208
[25]	validation_0-rms

In [ ]:
import xgboost

model = xgboost.XGBRegressor(n_estimators=1000, learning_rate=0.1, max_depth=14, random_state = 2022, colsample_bytree = 0.75, min_child_weight = 1)
model.fit(X_train, y_train)
y_pred = model.predict(X_val)

[06:26:21] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [ ]:
mse = mean_squared_error(y_val, y_pred)
rmse = np.sqrt(mse)

print('MSE : {0:.3f}, RMSE : {1:.3F}'.format(mse , rmse))
print('Variance score : {0:.3f}'.format(r2_score(y_val,y_pred)))

MSE : 0.005, RMSE : 0.071
Variance score : 0.660


In [ ]:
xgb_model = xgboost.XGBRegressor()

# 후보 파라미터 선정
params = {'max_depth':[14, 16], 'min_child_weight':[0.5, 1], 'colsample_bytree':[0.75, 1], 'learning_rate' : [0.2, 0.3]}
#{'colsample_bytree': 0.75, 'learning_rate': 0.2, 'max_depth': 16, 'min_child_weight': 0.5}

# gridsearchcv 객체 정보 입력(어떤 모델, 파라미터 후보, 교차검증 몇 번)
gridcv = GridSearchCV(xgb_model, param_grid=params, cv=3)

# 파라미터 튜닝 시작
gridcv.fit(X_train, y_train, early_stopping_rounds=30, eval_metric='rmse', eval_set=[(X_val, y_val)])

#튜닝된 파라미터 출력
print(gridcv.best_params_)

[07:02:43] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation_0-rmse:0.16614
Will train until validation_0-rmse hasn't improved in 30 rounds.
[1]	validation_0-rmse:0.143931
[2]	validation_0-rmse:0.127534
[3]	validation_0-rmse:0.115529
[4]	validation_0-rmse:0.107537
[5]	validation_0-rmse:0.101897
[6]	validation_0-rmse:0.097036
[7]	validation_0-rmse:0.093635
[8]	validation_0-rmse:0.091178
[9]	validation_0-rmse:0.089405
[10]	validation_0-rmse:0.088316
[11]	validation_0-rmse:0.087447
[12]	validation_0-rmse:0.086865
[13]	validation_0-rmse:0.086449
[14]	validation_0-rmse:0.086222
[15]	validation_0-rmse:0.08585
[16]	validation_0-rmse:0.085524
[17]	validation_0-rmse:0.085128
[18]	validation_0-rmse:0.084878
[19]	validation_0-rmse:0.084711
[20]	validation_0-rmse:0.084565
[21]	validation_0-rmse:0.08436
[22]	validation_0-rmse:0.084173
[23]	validation_0-rmse:0.084017
[24]	validation_0-rmse:0.083886
[25]	validation_0-rms

In [ ]:
import xgboost

model = xgboost.XGBRegressor(n_estimators=1000, learning_rate=0.2, max_depth=14, random_state = 2022, colsample_bytree = 0.75, min_child_weight = 0.5)
model.fit(X_train, y_train)
y_pred = model.predict(X_val)

[23:51:28] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [ ]:
mse = mean_squared_error(y_val, y_pred)
rmse = np.sqrt(mse)

print('MSE : {0:.3f}, RMSE : {1:.3F}'.format(mse , rmse))
print('Variance score : {0:.3f}'.format(r2_score(y_val,y_pred)))

MSE : 0.005, RMSE : 0.072
Variance score : 0.651


## test

In [ ]:
xbg_preds_test = model.predict(test_data.to_numpy())

In [ ]:
respec_final.loc[(respec_final['credit_score'].isnull()), 'credit_score'] = xbg_preds_test.round()

In [ ]:
respec_final.to_csv('/content/drive/MyDrive/2022_Bigcontest/Data/respec_final_XGB.csv')

##4) 교차 검증

In [ ]:
from sklearn.datasets import load_wine
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
import pandas as pd
import numpy as np

features = respec_train.drop(['credit_score'], axis = 1).to_numpy()
target = respec_train['credit_score'].to_numpy()
model = xgboost.XGBRegressor(n_estimators=1000, learning_rate=0.2, max_depth=14, random_state = 2022, colsample_bytree = 0.75, min_child_weight = 0.5)

kfold = KFold(n_splits=5)
accuracy_list = []

In [ ]:
iter_num = 0

for train_index, test_index in kfold.split(features):
    x_train, x_test = features[train_index], features[test_index]
    y_train, y_test = target[train_index], target[test_index]


    model.fit(x_train, y_train)
    predict = model.predict(x_test)
    iter_num += 1

    # 정확도 측정
    accuracy = np.round(accuracy_score(y_test, predict), 4)
    train_size = x_train.shape[0]
    test_size = x_test.shape[0]
    print('============================')
    print('iter 횟수 : ', iter_num)
    print('교차 검증 정확도 : ', accuracy)
    print('학습 데이터 크기 : ', train_size)
    print('검증 데이터 크기 : ', test_size)
    print('============================')
    print('iter 횟수 : ', iter_num)
    print('검증 셋트 인덱스 : ', test_index)
    print('============================')
    accuracy_list.append(accuracy)

print('============================')
print('평균 검증 정확도', np.mean(accuracy_list))